# Airport queuing system

#### We are simulating a simplified airport security system at a busy airport. 

#### Passengers arrive according to a Poisson distribution with $\lambda_1 = 50$ per minute (i.e., mean interarrival rate $\mu_1 = 0.2$ minutes) to the ID/boarding-pass check queue, where there are several servers who each have exponential service time with mean rate $\mu_2 = 0.75$ minutes. 

#### After that, the passengers are assigned to the shortest of the several personal-check queues, where they go through the personal scanner (time is uniformly distributed between 0.5 minutes and 1 minute).

#### We want to keep average wait times below 15 minutes, so there is a penality if a passenger waiting time is longer than 15 min. In addition, there is a cost associated with each ID/boarding-pass checker and personal scanner. We want to minimise 

## Process steps

In [1]:
# 1. Arrive at the airport security system
# 2. Get in the ID/boarding-pass check queue, and wait for a server.
# 3. Get the ID/boarding-pass checked from the server.
# 4. Get in scanners queue - the shortest
# 5. Go through the personal scanner.
# 6. Leave the airport security system area

## Setting Up the Environment

In [2]:
# ---------- Import modules -----------
import simpy
import random
import statistics
import pandas as pd
import numpy as np

In [3]:
# ------------ Set constants ---------------
num_checkers = 3 # number of boarding-pass checkers
num_scanners = 3 # number of scanners

arr_rate = 15 # arrival rate (passengers per minute)
check_rate = 0.75 # boarding-pass check rate (minutes per passenger)
min_scan = 0.5 # scanner minimum time for uniform distribution
max_scan = 1.0 # scanner maximum time for uniform distribution
run_time = 720 # run time (minutes) per simulation

In [4]:
def calculate_exceeding_time(minutes):
    if minutes <= 15:
        return 0
    else:
        exceeding_time = minutes - 15
        return exceeding_time

## Creating the Model

In [5]:
# System class

class System(object):
    def __init__(self,env):
        self.env = env
        self.checker = simpy.Resource(env,num_checkers) # define number of boarding-pass checkers
        self.scanner = [] # define a set of scanners with 1 each; needed because each has its own queue
        for i in range(num_scanners):
            self.scanner.append(simpy.Resource(env,1))

    # define boarding-pass check time (exponential)
    def check(self,passenger):
        # For some reason in python, expovariate actually uses 1 over the mean, like Poisson
        yield self.env.timeout(random.expovariate(1.0/check_rate))

    # define scan time (uniform)
    def scan(self,passenger):
        yield self.env.timeout(random.uniform(min_scan,max_scan))

# Passenger process through system

def passenger(env,name,s):

    # access global variables to be able to modify them
    global check_wait
    global scan_wait
    global sys_time
    global wait_time
    global wait_cost
    global tot_through

    time_arrive = env.now # note arrival time of passenger


    # print('%s arrives at time %.2f' % (name,timeArrive))

    # Go through boarding-pass check queue
    with s.checker.request() as request:
        # print('check queue length = %d' % len(s.checker.queue))
        yield request # request a checker
        t_in = env.now # note when passenger starts being checked
        yield env.process(s.check(name)) # call check process
        t_out = env.now # note when passenger ends being checked
        check_time.append(t_out - t_in) # calculate total time for passenger to be checked

    # Find the shortest scanner queue (note: scanners are numbered 0 through numScanners-1)
    minq = 0
    for i in range(1,num_scanners):
        if (len(s.scanner[i].queue) < len(s.scanner[minq].queue)):
            minq = i

    # print('scanner queue %d lengths = %d' % (minq,len(s.scanner[minq].queue)))

    # Go through scanner queue
    with s.scanner[minq].request() as request: # use scanner number minq (the shortest, from above)
        yield request # request the scanner
        t_in = env.now # note when passenger starts being scanned
        yield env.process(s.scan(name)) # call scan process
        t_out = env.now # note when passenger ends being scanned
        scan_time.append(t_out - t_in) # calculate total time for passenger to be scanned
          
    time_leave = env.now # note time passenger finishes
    sys_time.append(time_leave - time_arrive) # calculate total time in system for passenger
    wait_time.append(sys_time[tot_through]-check_time[tot_through]-scan_time[tot_through])
    wait_cost.append(calculate_exceeding_time(wait_time[tot_through])*0.1)
    tot_through += 1 # count another passenger who got through the system


# Passenger arrival process

def setup(env):
    i = 0
    s = System(env)
    while True: # keep doing it (until simulation ends)
        yield env.timeout(random.expovariate(arr_rate)) # find tieme until next passenger is created
        i += 1 # count one more passenger

        # send the passenger through its process
        env.process(passenger(env,'Passenger %d' % i,s)) # name the passenger "Passenger i"
        


## Run the Model

In [6]:
min_checkers = 10
max_checkers = 13
min_scanners = 10
max_scanners = 13
n_0 = 139 # number of replications

results = []

for i in range(min_checkers, max_checkers,1):
    for j in range(min_scanners, max_scanners,1):
        l = []
        random.seed(451)
        for k in range(n_0):            
            env = simpy.Environment()

            # initialize global variables
            tot_through = 0
            check_time = []
            scan_time = []
            sys_time = []
            wait_time = []
            wait_cost = []
            num_checkers = i
            num_scanners = j
            
            # run the simulation
            env.process(setup(env)) # start passenger arrival process
            env.run(until=run_time) # run for runTime simulated minutes             
            l.append(round((round(sum(wait_cost[1:tot_through]), 2) + (num_checkers + num_scanners)*(run_time/60)*150)/1000,2))
        results.append(l)
#             print('checkers: %d , scanners: %d , passengers %d, replication %d' % (num_checkers, num_scanners, tot_through, k+1))


In [7]:
print(results)
X = np.array(results)

[[65.28, 70.77, 60.25, 68.15, 66.25, 63.88, 65.26, 63.72, 62.52, 65.29, 61.6, 69.04, 64.2, 62.33, 59.46, 69.04, 61.41, 62.56, 59.7, 63.57, 61.57, 58.99, 55.68, 69.41, 65.88, 61.82, 65.39, 67.87, 69.34, 64.47, 60.51, 65.72, 67.08, 61.64, 62.87, 63.37, 60.02, 65.64, 68.49, 66.02, 61.28, 58.75, 62.45, 60.51, 65.66, 64.46, 60.75, 62.05, 64.68, 68.23, 63.12, 73.6, 64.78, 69.23, 59.43, 70.99, 64.92, 67.5, 67.32, 66.57, 67.4, 59.83, 63.44, 54.52, 65.71, 66.27, 68.82, 62.96, 71.22, 57.38, 60.96, 68.87, 68.24, 75.03, 57.99, 64.7, 66.81, 65.49, 64.77, 63.04, 58.33, 70.38, 65.07, 61.65, 62.39, 63.25, 67.49, 63.95, 61.07, 58.48, 64.53, 64.85, 64.52, 64.6, 70.21, 61.34, 65.61, 62.13, 62.4, 63.83, 68.09, 60.51, 68.4, 59.15, 66.12, 70.45, 61.21, 64.08, 58.25, 62.55, 64.59, 67.29, 60.87, 64.14, 61.69, 67.08, 63.57, 65.19, 67.2, 75.22, 54.52, 62.91, 66.88, 62.8, 62.55, 66.13, 62.82, 59.24, 59.37, 57.86, 63.65, 65.0, 66.71, 61.68, 59.07, 67.52, 61.03, 62.02, 62.41], [67.99, 67.07, 58.41, 62.85, 60.52, 6

In [43]:
corr_coef = np.corrcoef(X[4], X[1])[0, 1]
print(corr_coef)

0.023516112570704167


In [44]:
def sample_variance(X):
    n = len(X[0])
    k = len(X)    
    som = 0
    for i in range(n):
        for j in range(k):
            t = np.subtract(X[j,i]-np.mean(X[:,i]),np.mean(X[j,:]))+np.mean(X)
            som += 2*t**2
    return som/(k-1)/(n-1)

In [45]:
print('Sample variance:')
print(sample_variance(X))

print('To find g value:')
print('(k-1) is:')
print(len(X)-1)
print('(k-1)*(n0-1) is:')
print((len(X)-1)*(len(X[0])-1))

Sample variance:
21.637971111111117
To find g value:
(k-1) is:
8
(k-1)*(n0-1) is:
32


In [49]:
d = 1
g = 2.89


def find_N(X,g,d):
    S2 = sample_variance(X)
    ans = max([len(X[0]),g**2*S2/d**2])
    return ans

find_N(X,g,d)

180.72249851711115

In [8]:
np.mean(X,axis=1)

array([64.1128777 , 64.03172662, 64.54848921, 63.73266187, 42.41604317,
       42.92546763, 64.82366906, 42.24633094, 43.2       ])